In [1]:
import os.path
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import pandas as pd
import dlib
import numpy as np
import copy
import pickle

Loading mode, get test_img names etc.

In [2]:
# input labels.
label = pd.read_csv("/home/clydechx/FairFace/mydataset/lfw/test_labels.csv")
imgs_path = "/home/clydechx/FairFace/mydataset/lfw/lfw"
img_names = []

# 遍历每个文件夹
for index, row in label.iterrows():
    # Get the file path
    file_path = row['file']

    # Use os.path.basename to get the file name without the directory
    # Add the file name to the img_names list
    img_names.append(file_path)
print(img_names)
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

model_fair_7 = torchvision.models.resnet34(pretrained=True)
model_fair_7.fc = nn.Linear(model_fair_7.fc.in_features, 6)
# if runs on a machine without GPU, should do map_location=torch.device('cpu')
#model_fair_7.load_state_dict(torch.load('fair_face_models/fairface_alldata_20191111.pt')) 
model_fair_7.load_state_dict(torch.load('/home/clydechx/FairFace/LFWA+_model.pt'))
model_fair_7 = model_fair_7.to(device)


# trans

trans = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

['John_Paul_II\\John_Paul_II_0004.jpg', 'Abdel_Nasser_Assidi\\Abdel_Nasser_Assidi_0001.jpg', 'Padraig_Harrington\\Padraig_Harrington_0004.jpg', 'Orlando_Bloom\\Orlando_Bloom_0003.jpg', 'Jon_Gruden\\Jon_Gruden_0007.jpg', 'Amelie_Mauresmo\\Amelie_Mauresmo_0005.jpg', 'Pedro_Almodovar\\Pedro_Almodovar_0005.jpg', 'Kurt_Warner\\Kurt_Warner_0001.jpg', 'Dave_Johnson\\Dave_Johnson_0001.jpg', 'Jessica_Lange\\Jessica_Lange_0002.jpg', 'Naoto_Kan\\Naoto_Kan_0003.jpg', 'E_Clay_Shaw\\E_Clay_Shaw_0001.jpg', 'Hu_Jintao\\Hu_Jintao_0005.jpg', 'Richard_Gere\\Richard_Gere_0002.jpg', 'Marcelo_Ebrard\\Marcelo_Ebrard_0002.jpg', 'Steven_Craig\\Steven_Craig_0001.jpg', 'Richard_Harris\\Richard_Harris_0001.jpg', 'Glen_DaSilva\\Glen_DaSilva_0001.jpg', 'Jim_Furyk\\Jim_Furyk_0005.jpg', 'Serena_Williams\\Serena_Williams_0017.jpg', 'George_W_Bush\\George_W_Bush_0106.jpg', 'Jack_Straw\\Jack_Straw_0002.jpg', 'James_Roberts\\James_Roberts_0001.jpg', 'Albert_Costa\\Albert_Costa_0002.jpg', 'Antonio_Palocci\\Antonio_Palocci

/home/clydechx/FairFace/env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/clydechx/FairFace/env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Start running the model.

In [3]:
# # Define a hook for all ReLU layers to compute the sum of activations of each filter
# activation = {}
# def relu_hook(module, input, output, layer_name):
#     activation[layer_name] = output.detach()
# # Register the hook for all ReLU layers
# for layer_name, layer in model.named_modules():
#     if isinstance(layer, torch.nn.ReLU):
#         layer.register_forward_hook(relu_hook, layer_name=layer_name)
num_filters = {}
activation = {}
filter_sums = {}
def get_activation(name):
    def hook(model, input, output):
        # Squeeze the tensor to remove the dimension with size 1
        squeezed_tensor = torch.squeeze(output.detach())
        activation[name] = squeezed_tensor
        # If squeezed_tensor has more than one dimension, flatten it
        if len(squeezed_tensor.shape) > 1:
            flattened_tensor = torch.flatten(squeezed_tensor, start_dim=1)
        else:
            flattened_tensor = squeezed_tensor
        # Sum along the second dimension if it exists, otherwise keep the tensor as is
        if flattened_tensor.dim() > 1:
            summed_tensor = flattened_tensor.sum(dim=1)
        else:
            summed_tensor = flattened_tensor
        # Convert the tensor to a list
        summed_list = summed_tensor.tolist()
        filter_sums[name] = summed_list
        num_filters[name] = len(summed_list)
    return hook

for name, layer in model_fair_7.named_modules():
    if isinstance(layer, torch.nn.ReLU):
        layer.register_forward_hook(get_activation(name))
    

In [6]:
# Define class labels for your model
classes = ['White+Female', 'White+Male', 'Asian+Female', 'Asian+Male', 'Black+Female', 'Black+Male', 'Indian+Female', 'Indian+Male']
# Define variables to track accuracy and class-wise accuracy
correct = 0
total = 0
class_correct = [0] * 8
class_total = [0] * 8

# Updated race and gender dictionaries
race_dict = {'White': 0, 'Asian': 1, 'Black': 2, 'Indian': 3}
gender_dict = { 'Male': 0, 'Female': 1}

# New class map
my_class_map = {
    (0, 1): 0,
    (0, 0): 1,
    (1, 1): 2,
    (1, 0): 3,
    (2, 1): 4,
    (2, 0): 5,
    (3, 1): 6,
    (3, 0): 7,
}

image_predictation_result = {cl: {} for cl in range(len(classes))}
activation_all_images = []
filtersum_all_images = []
miss_predicated_images = []
# Set the model to evaluation mode
model_fair_7.eval()
my_label = label
for index, img_name in enumerate(img_names):
    if index % 1000 == 0:
        print("Predicting... {}/{}".format(index, len(img_names)))
    img_name_in_label =  img_name
    label_row = my_label[my_label['file'] == img_name_in_label].iloc[0]
    
    # load image, run model.
    new_name = img_name.split('\\')[-2] + '/' +img_name.split('\\')[-1] 
    loading_path =  os.path.join(imgs_path,new_name)
    image = dlib.load_rgb_image(loading_path)
    image = trans(image)
    image = image.view(1, 3, 224, 224)  # reshape image to match model dimensions (1 batch size)
    image = image.to(device)
    
    outputs = model_fair_7(image)
    outputs = outputs.cpu().detach().numpy()
    outputs = np.squeeze(outputs)
    
    race_outputs = outputs[:4]
    gender_outputs = outputs[4:6]
    
    race_score = np.exp(race_outputs) / np.sum(np.exp(race_outputs))
    gender_score = np.exp(gender_outputs) / np.sum(np.exp(gender_outputs))

    race_pred = np.argmax(race_score)
    gender_pred = np.argmax(gender_score)

    predict_class = my_class_map[(race_pred,gender_pred)]
    label_class_name = label_row['race']+"+" + label_row['gender']
    label_class = classes.index(label_class_name)
    
    
    total +=1
    class_total[label_class] += 1
    image_predictation_result[label_class ][index] = predict_class
    filtersum_all_images.append(copy.deepcopy(filter_sums))
    
    # predict correctly 
    if predict_class == label_class:
        correct+=1
        class_correct[label_class]+=1
    # predict wrongly
    else:
        miss_predicated_images.append(index)
        print("Predicted label: {}, Ground truth: {}".format(predict_class, label_class))


Predicting... 0/6880
Predicted label: 1, Ground truth: 7
Predicted label: 7, Ground truth: 1
Predicted label: 3, Ground truth: 1
Predicted label: 1, Ground truth: 0
Predicted label: 1, Ground truth: 3
Predicted label: 7, Ground truth: 3
Predicted label: 1, Ground truth: 2
Predicted label: 7, Ground truth: 3
Predicted label: 1, Ground truth: 0
Predicted label: 1, Ground truth: 3
Predicted label: 0, Ground truth: 3
Predicted label: 3, Ground truth: 1
Predicted label: 7, Ground truth: 3
Predicted label: 7, Ground truth: 3
Predicted label: 2, Ground truth: 0
Predicted label: 7, Ground truth: 1
Predicted label: 1, Ground truth: 3
Predicted label: 2, Ground truth: 1
Predicted label: 5, Ground truth: 1
Predicted label: 7, Ground truth: 1
Predicted label: 3, Ground truth: 1
Predicted label: 2, Ground truth: 3
Predicted label: 7, Ground truth: 1
Predicted label: 0, Ground truth: 1
Predicted label: 7, Ground truth: 2
Predicted label: 0, Ground truth: 1
Predicted label: 7, Ground truth: 1
Predict

Predicted label: 3, Ground truth: 1
Predicted label: 7, Ground truth: 1
Predicted label: 7, Ground truth: 3
Predicted label: 3, Ground truth: 2
Predicted label: 0, Ground truth: 1
Predicted label: 5, Ground truth: 1
Predicted label: 1, Ground truth: 3
Predicted label: 4, Ground truth: 1
Predicted label: 0, Ground truth: 1
Predicted label: 1, Ground truth: 3
Predicted label: 7, Ground truth: 3
Predicted label: 6, Ground truth: 4
Predicted label: 0, Ground truth: 2
Predicted label: 5, Ground truth: 4
Predicted label: 7, Ground truth: 1
Predicted label: 7, Ground truth: 3
Predicted label: 1, Ground truth: 3
Predicted label: 1, Ground truth: 3
Predicted label: 0, Ground truth: 1
Predicted label: 4, Ground truth: 2
Predicted label: 1, Ground truth: 3
Predicted label: 3, Ground truth: 0
Predicted label: 0, Ground truth: 3
Predicted label: 3, Ground truth: 7
Predicted label: 0, Ground truth: 1
Predicted label: 1, Ground truth: 3
Predicted label: 1, Ground truth: 3
Predicted label: 0, Ground t

In [7]:
# Print the overall accuracy and accuracy for each class
print('size of test set: {}, accuracy of the network on the test set: {:.2f}%'.format(total, 100 * correct / total))
for i in range(8):
    print('Accuracy of {} : {:.0f}% of {} images'.format(classes[i], 100 * class_correct[i] / class_total[i], class_total[i]))
print("miss_predicated_images num = {}".format(len(miss_predicated_images)))

size of test set: 6880, accuracy of the network on the test set: 74.16%
Accuracy of White+Female : 72% of 1027 images
Accuracy of White+Male : 89% of 4165 images
Accuracy of Asian+Female : 52% of 333 images
Accuracy of Asian+Male : 31% of 1003 images
Accuracy of Black+Female : 51% of 82 images
Accuracy of Black+Male : 56% of 172 images
Accuracy of Indian+Female : 22% of 18 images
Accuracy of Indian+Male : 51% of 80 images
miss_predicated_images num = 1778


In [8]:
print(len(activation_all_images))
# print(len(activation_all_images[0]))
# print(len(activation_all_images[0]["features.22"][0]))
hooked_layer_names = list(activation.keys())
print(hooked_layer_names)
print(len(filtersum_all_images[0]))
print((filtersum_all_images[0]))

0
['relu', 'layer1.0.relu', 'layer1.1.relu', 'layer1.2.relu', 'layer2.0.relu', 'layer2.1.relu', 'layer2.2.relu', 'layer2.3.relu', 'layer3.0.relu', 'layer3.1.relu', 'layer3.2.relu', 'layer3.3.relu', 'layer3.4.relu', 'layer3.5.relu', 'layer4.0.relu', 'layer4.1.relu', 'layer4.2.relu']
17
{'relu': [5396.716796875, 2841.75732421875, 4123.06787109375, 5295.45361328125, 2099.715576171875, 3622.589111328125, 2002.789306640625, 9778.529296875, 8589.5791015625, 1225.8187255859375, 3085.568115234375, 4695.3076171875, 1521.036376953125, 2965.94921875, 4040.20849609375, 0.0, 0.0, 3128.3603515625, 2565.34228515625, 628.2861328125, 5776.47705078125, 4022.79736328125, 3070.223388671875, 305.0179443359375, 2870.151123046875, 1919.134033203125, 6368.3310546875, 7310.8447265625, 5951.765625, 2992.34765625, 7744.3896484375, 0.0, 2935.607421875, 4759.61328125, 583.6227416992188, 0.0, 7462.99267578125, 4171.3662109375, 6640.755859375, 3516.640380859375, 3726.63232421875, 0.0, 3506.75537109375, 1476.29833984

In [9]:
def save_intermediate_data(data, file_path):
    """
    Save intermediate data to a file using pickle serialization.

    Args:
        data: The data to be saved.
        file_path (str): The path of the file to save the data.
    """
    with open(file_path, 'wb') as file:
        pickle.dump(data, file)
# save intermediate results
save_intermediate_data(activation_all_images, "activation_CIFAR10_VGG16.pkl")
save_intermediate_data(image_predictation_result, "image_predictation_result.pkl")
save_intermediate_data(miss_predicated_images, "miss_predicated_images.pkl")
save_intermediate_data(hooked_layer_names, "hooked_layer_names.pkl")
save_intermediate_data(filtersum_all_images, "filtersum_all_images.pkl")
save_intermediate_data(num_filters, "num_filters.pkl")

In [10]:
def load_intermediate_data(file_path):
    """
    Load intermediate data from a file.

    Args:
        file_path (str): The path of the file to load the data from.

    Returns:
        The loaded data.
    """
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    return data


activation_all_images = load_intermediate_data("activation_CIFAR10_VGG16.pkl")
image_predictation_result = load_intermediate_data("image_predictation_result.pkl")
miss_predicated_images = load_intermediate_data("miss_predicated_images.pkl")
hooked_layer_names = load_intermediate_data("hooked_layer_names.pkl")
filtersum_all_images = load_intermediate_data("filtersum_all_images.pkl")
num_filters = load_intermediate_data("num_filters.pkl")

In [11]:
num_test_images = len(filtersum_all_images)
def find_outliers_zscore(data, threshold=1):
    """
    Find outliers in a list of values using z-score method.

    Args:
        data (list or numpy array): List of numeric values.
        threshold (float, optional): Z-score threshold for identifying outliers. Default is 3.

    Returns:
        list: List of outlier values.
    """
    # Convert data to numpy array
    data = np.array(data)
    if np.std(data) == 0:
        return []
    # Calculate z-scores
    z_scores = (data - np.mean(data)) / np.std(data)
    # Find outliers based on the threshold
    # outliers = data[np.abs(z_scores) > threshold]
    outliers = np.where(z_scores < -threshold)[0]
    return outliers

In [12]:
# find outlier for all mis_predicated images:
# cat (3)
class_id = 3
all_image_class = image_predictation_result[class_id].keys()
wrong_images = [x for x in all_image_class if image_predictation_result[class_id][x] != class_id]
print("num of wrong images in class {} is {}".format(class_id, len(wrong_images)))

num of wrong images in class 3 is 693


In [13]:
z_score_threshold = 1.0

for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    for image in wrong_images:
        # print("image # {}".format(image))
        outliers = find_outliers_zscore(filtersum_all_images[image][layer_name], threshold=z_score_threshold)
        if len(outliers) > 0:
            print("image # {}, layer # {} has outliers {}".format(image, layer_name, outliers))



layer relu
image # 21, layer # relu has outliers [15 16 19 31 34 35 41 45 46 47 48 49 57 60 63]
image # 23, layer # relu has outliers [15 16 19 31 34 35 41 45 46 47 48 49 57 60 63]
image # 28, layer # relu has outliers [15 16 23 31 35 41 46 47 48 49 57 60 63]
image # 32, layer # relu has outliers [15 16 23 31 35 41 45 46 47 48 49 57 63]
image # 33, layer # relu has outliers [15 16 19 23 31 34 35 41 45 46 47 48 49 50 57 60 63]
image # 47, layer # relu has outliers [15 16 31 34 35 41 45 46 47 48 49 57 60 63]
image # 49, layer # relu has outliers [15 16 23 31 34 35 41 45 46 47 48 49 57 60 63]
image # 60, layer # relu has outliers [15 16 19 31 34 35 41 45 46 47 48 49 57 60 63]
image # 78, layer # relu has outliers [15 16 19 31 34 35 41 45 46 47 48 49 57 60 63]
image # 97, layer # relu has outliers [15 16 19 31 34 35 41 45 46 47 49 57 60 63]
image # 99, layer # relu has outliers [15 16 23 31 34 35 41 45 46 47 48 49 57 60 63]
image # 110, layer # relu has outliers [15 16 19 31 34 35 41 45 46

## activation level for the whole test set


In [ ]:
print(num_filters)

{'relu': 64, 'layer1.0.relu': 64, 'layer1.1.relu': 64, 'layer1.2.relu': 64, 'layer2.0.relu': 128, 'layer2.1.relu': 128, 'layer2.2.relu': 128, 'layer2.3.relu': 128, 'layer3.0.relu': 256, 'layer3.1.relu': 256, 'layer3.2.relu': 256, 'layer3.3.relu': 256, 'layer3.4.relu': 256, 'layer3.5.relu': 256, 'layer4.0.relu': 512, 'layer4.1.relu': 512, 'layer4.2.relu': 512}


In [ ]:
# outliers for each layer for each image
z_score_threshold = 2
all_outliers_for_image = {x: {} for x in hooked_layer_names}
num_times_being_outlier = {}

for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    num_times_being_outlier[layer_name] = [0] * num_filters[layer_name]
    for image in range(num_test_images):
        all_outliers_for_image[layer_name][image] = find_outliers_zscore(filtersum_all_images[image][layer_name], threshold=z_score_threshold)
        for k in all_outliers_for_image[layer_name][image]:
            num_times_being_outlier[layer_name][k] += 1

layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu


In [ ]:
non_activation_ratio_whole_data = {}
ratio_threshold = 0.01
for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    s = {x: num_times_being_outlier[layer_name][x] / num_test_images for x in range(num_filters[layer_name]) if num_times_being_outlier[layer_name][x] > 0}
    non_activation_ratio_whole_data[layer_name] = {k: v for k, v in s.items() if v > ratio_threshold}
    # non_activation_ratio_whole_data[layer_name] = [x/num_test_images for x in num_times_being_outlier[layer_name]]



layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu


In [ ]:
for layer_name in hooked_layer_names:
    print(layer_name, non_activation_ratio_whole_data[layer_name])



relu {}
layer1.0.relu {}
layer1.1.relu {}
layer1.2.relu {39: 0.028117582618221653}
layer2.0.relu {}
layer2.1.relu {1: 0.011411356582070476}
layer2.2.relu {60: 0.013511046193171445, 105: 0.01314588278254519}
layer2.3.relu {105: 0.034873105714807374}
layer3.0.relu {7: 0.012415555961292679, 22: 0.023096585722110645, 144: 0.013237173635201752, 156: 0.01031586635019171, 231: 0.08864341792952346, 241: 0.05513967500456454, 251: 0.02300529486945408}
layer3.1.relu {18: 0.020996896111009678, 111: 0.08188789483293774, 118: 0.07750593390542268, 144: 0.07960562351652364, 146: 0.15775059339054226, 151: 0.021179477816322803, 162: 0.027204674091656018, 180: 0.23114843892641956, 197: 0.3521088186963666, 205: 0.012598137666605806, 208: 0.01077232061347453, 213: 0.04035055687420121, 229: 0.016341062625524923, 231: 0.05723936461566551, 236: 0.10744933357677561, 251: 0.16359320796056234}
layer3.2.relu {18: 0.02683951068102976, 118: 0.09905057513237174, 130: 0.013054591929888625, 151: 0.025287566185868176, 

## non-activation times ratio for different classes, for right/wrong

In [ ]:
# outliers for each layer for each image
z_score_threshold = 2
all_outliers_for_miss_predicated_image = {x: {} for x in hooked_layer_names}
num_times_being_outlier_miss_predicated_images = {}

for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    num_times_being_outlier_miss_predicated_images[layer_name] = [0] * num_filters[layer_name]
    for image in wrong_images:
        all_outliers_for_miss_predicated_image[layer_name][image] = find_outliers_zscore(filtersum_all_images[image][layer_name], threshold=z_score_threshold)
        for k in all_outliers_for_miss_predicated_image[layer_name][image]:
            num_times_being_outlier_miss_predicated_images[layer_name][k] += 1

layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu


In [ ]:
non_activation_ratio_miss_predicated_image = {}
num_miss_predicated_images = len(wrong_images)

for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    s = {x: num_times_being_outlier_miss_predicated_images[layer_name][x] / num_miss_predicated_images for x in range(num_filters[layer_name]) if num_times_being_outlier_miss_predicated_images[layer_name][x] > 0}
    non_activation_ratio_miss_predicated_image[layer_name] = {k: v for k, v in s.items() if v > ratio_threshold}


for layer_name in hooked_layer_names:
    print(layer_name, non_activation_ratio_miss_predicated_image[layer_name])



layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu
relu {}
layer1.0.relu {}
layer1.1.relu {}
layer1.2.relu {39: 0.027131782945736434}
layer2.0.relu {}
layer2.1.relu {1: 0.011627906976744186}
layer2.2.relu {31: 0.011627906976744186, 60: 0.015503875968992248, 105: 0.01937984496124031}
layer2.3.relu {105: 0.031007751937984496}
layer3.0.relu {7: 0.011627906976744186, 22: 0.04263565891472868, 144: 0.011627906976744186, 231: 0.05426356589147287, 241: 0.08914728682170543, 251: 0.015503875968992248}
layer3.1.relu {18: 0.015503875968992248, 111: 0.050387596899224806, 118: 0.07751937984496124, 139: 0.011627906976744186, 144: 0.046511627906976744, 146: 0.12015503875968993, 151: 0.011627906976744186, 162: 0.031007751937984

## non-activation times for different classes

In [ ]:
all_outliers_for_classes = {x: {y: {} for y in range(len(classes))} for x in hooked_layer_names}
num_times_being_outlier_classes = {y: {} for y in range(len(classes))}

for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    for cla in range(len(classes)):
        num_times_being_outlier_classes[cla][layer_name] = [0] * num_filters[layer_name]
        for image in image_predictation_result[cla]:
            all_outliers_for_classes[layer_name][cla][image] = find_outliers_zscore(filtersum_all_images[image][layer_name], threshold=z_score_threshold)
            for k in all_outliers_for_classes[layer_name][cla][image]:
                num_times_being_outlier_classes[cla][layer_name][k] += 1



layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu


In [ ]:
non_activation_ratio_classes = {y: {} for y in range(len(classes))}
ratio_threshold = 0.01
for cla in range(len(classes)):
    for layer_name in hooked_layer_names:
        print("layer {}".format(layer_name))
        s = {x: num_times_being_outlier_classes[cla][layer_name][x] / len(image_predictation_result[cla]) for x in range(num_filters[layer_name]) if num_times_being_outlier_classes[cla][layer_name][x] > 0}
        non_activation_ratio_classes[cla][layer_name] = {k: v for k, v in s.items() if v > ratio_threshold}



layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu
layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu
layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu
layer r

In [ ]:
# for cla in range(len(classes)):
#     for layer_name in hooked_layer_names:
#         print(layer_name, non_activation_ratio_classes[cla][layer_name])

# less non-activation classes: 0, 2, 4, 8
# many: 1, 3, 5, 6, 7, 9
cla = 6
print('Accuracy of {} : {:.0f}% of {} images'.format(classes[cla], 100 * class_correct[cla] / class_total[cla], class_total[cla]))
for layer_name in hooked_layer_names:
    print(layer_name, non_activation_ratio_classes[cla][layer_name])


Accuracy of Middle Eastern+Male : 68% of 813 images
relu {}
layer1.0.relu {}
layer1.1.relu {}
layer1.2.relu {39: 0.02706027060270603}
layer2.0.relu {}
layer2.1.relu {1: 0.024600246002460024, 80: 0.01107011070110701}
layer2.2.relu {105: 0.017220172201722016}
layer2.3.relu {76: 0.012300123001230012, 105: 0.05289052890528905}
layer3.0.relu {7: 0.028290282902829027, 22: 0.03690036900369004, 118: 0.013530135301353014, 121: 0.015990159901599015, 122: 0.01968019680196802, 144: 0.012300123001230012, 156: 0.01845018450184502, 231: 0.0971709717097171, 241: 0.038130381303813035, 251: 0.028290282902829027}
layer3.1.relu {18: 0.01845018450184502, 111: 0.11193111931119311, 118: 0.14514145141451415, 144: 0.06888068880688807, 146: 0.2029520295202952, 151: 0.03075030750307503, 161: 0.017220172201722016, 162: 0.023370233702337023, 180: 0.22878228782287824, 197: 0.3997539975399754, 200: 0.012300123001230012, 213: 0.05781057810578106, 225: 0.015990159901599015, 229: 0.013530135301353014, 231: 0.0676506765